In [2]:
import pandas as pd
import numpy as np

In [81]:
##################
# COMPAS Cleanup #
##################

import datetime as dt

def date_to_ord(df):
    df = df.apply(lambda col: pd.to_datetime(col, errors='ignore')
                  if col.dtypes == object else col, axis=0)
    date_cols = df.select_dtypes(include='datetime64[ns]').columns

    for c in date_cols:
        df[c] = df[c].apply(lambda x: x.toordinal() if pd.notna(x) else np.nan) 
        col_min = df[c].min()
        df[c] = df[c].apply(lambda x: x - col_min if pd.notna(x) else np.nan)
    
    return df

def label_cat_nans(df):
    cat_cols = df.select_dtypes(include='object').columns
    
    for c in cat_cols:
        df[c] = df[c].apply(lambda x: c + '_missing' if pd.isna(x) else x)
    return df

# data is filtered to be equal to propublica's
df = pd.read_csv('datasets/compas-scores.csv')
# to_keep = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
#            'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']
# df = df[to_keep]
# df = df[(df.days_b_screening_arrest <= 30)
#         & (df.days_b_screening_arrest >= -30)
#         & (df.is_recid != -1)
#         & (df.c_charge_degree != "O")
#         & (df.score_text != 'N/A')
#        ]

# make dates ordinal
ord_df = date_to_ord(df)
# Remove nans from cat variables
ncn_df = label_cat_nans(ord_df)
# make pa binary
ncn_df.race = ncn_df.race.apply(lambda x: x if pd.isna(x)
                                else 1 if x == 'Caucasian' 
                                else 0) 

ncn_df.drop(columns=['id', 'name', 'first', 'last',
                     'c_case_number', 'r_case_number', 'vr_case_number',
                     'c_charge_desc', 'r_charge_desc', 'vr_charge_desc',
                     'type_of_assessment', 'v_type_of_assessment',
                     'r_charge_degree', 'vr_charge_degree', 'is_violent_recid',
                     'age_cat', 'dob', 'score_text', 'v_score_text', 'decile_score.1'
                    ], inplace=True)
ncn_df = ncn_df[ncn_df.is_recid != -1]

ncn_df.to_csv('datasets/compas_ne_new.csv', index=False)

In [ ]:
#########################
# German Credit Cleanup #
#########################

df = pd.read_csv('datasets/german.csv', header=None)

# Data needed column names
df.columns = ['checking', 'duration', 'credit_history', 'purpose', 'credit_amount',
              'savings', 'employment', 'inst_rate', 'personal_status', 'other_debtors',
              'residence_time', 'property', 'age', 'other_inst', 'housing', 'num_credits',
              'job', 'dependants', 'phone', 'foreigner', 'label']

# Turn criptic values into interpretable form
df = df.replace({'checking': {'A11': 'check_low', 'A12': 'check_mid', 'A13': 'check_high',
                              'A14': 'check_none'},
                 'credit_history': {'A30': 'debt_none', 'A31': 'debt_noneBank',
                                    'A32': 'debt_onSchedule','A33': 'debt_delay',
                                    'A34': 'debt_critical'},
                 'purpose': {'A40': 'pur_newCar', 'A41': 'pur_usedCar',
                             'A42': 'pur_furniture', 'A43': 'pur_tv',
                             'A44': 'pur_appliance', 'A45': 'pur_repairs',
                             'A46': 'pur_education', 'A47': 'pur_vacation',
                             'A48': 'pur_retraining', 'A49': 'pur_business',
                             'A410': 'pur_other'},
                 'savings': {'A61': 'sav_small', 'A62': 'sav_medium', 'A63': 'sav_large',
                             'A64': 'sav_xlarge', 'A65': 'sav_none'},
                 'employment': {'A71': 'emp_unemployed', 'A72': 'emp_lessOne',
                                'A73': 'emp_lessFour', 'A74': 'emp_lessSeven',
                                'A75': 'emp_moreSeven'},
                 'other_debtors': {'A101': 'debtor_none', 'A102': 'debtor_coApp',
                                   'A103': 'debtor_guarantor'},
                 'property': {'A121': 'prop_realEstate', 'A122': 'prop_agreement',
                              'A123': 'prop_car', 'A124': 'prop_none'},
                 'other_inst': {'A141': 'oi_bank', 'A142': 'oi_stores', 'A143': 'oi_none'},
                 'housing': {'A151': 'hous_rent', 'A152': 'hous_own', 'A153': 'hous_free'},
                 'job': {'A171': 'job_unskilledNR', 'A172': 'job_unskilledR',
                         'A173': 'job_skilled', 'A174': 'job_highSkill'},
                 'phone': {'A191': 0, 'A192': 1},
                 'foreigner': {'A201': 1, 'A202': 0},
                 'label': {2: 0}})

# More criptic values translating
df['status'] = np.where(df.personal_status == 'A91', 'divorced',
                        np.where(df.personal_status == 'A92', 'divorced', 
                                 np.where(df.personal_status == 'A93', 'single',
                                          np.where(df.personal_status == 'A95', 'single',
                                                   'married'))))
# Translate gender values
df['gender'] = np.where(df.personal_status == 'A92', 0,
                        np.where(df.personal_status == 'A95', 0,
                                 1))

# Drop personal_status column
df = df.drop(['personal_status'], axis=1)

# Drop 10% of numerical values
num_cols = list(df.select_dtypes(include='int64').columns)
num_cols.remove('label')
num_cols.remove('gender')
a = np.ones(9000, dtype=int)
a[:900] = 0
np.random.shuffle(a)
a = a.astype(bool).reshape((1000, 9))
dropped = df[num_cols].where(a)
cats = df.drop(columns=num_cols)
df = pd.concat([cats, dropped], axis=1)

df.to_csv('datasets/german_ne.csv', index=False)

In [3]:
df = pd.read_csv('datasets/german_ne.csv')
df.head()

,checking,credit_history,purpose,savings,employment,other_debtors,property,other_inst,housing,job,...,gender,duration,credit_amount,inst_rate,residence_time,age,num_credits,dependants,phone,foreigner
0,check_low,debt_critical,pur_tv,sav_none,emp_moreSeven,debtor_none,prop_realEstate,oi_none,hous_own,job_skilled,...,1,6.0,1169.0,4.0,4.0,67.0,2.0,1.0,1.0,1.0
1,check_mid,debt_onSchedule,pur_tv,sav_small,emp_lessFour,debtor_none,prop_realEstate,oi_none,hous_own,job_skilled,...,0,48.0,5951.0,2.0,2.0,NaN,1.0,1.0,0.0,1.0
2,check_none,debt_critical,pur_education,sav_small,emp_lessSeven,debtor_none,prop_realEstate,oi_none,hous_own,job_unskilledR,...,1,12.0,2096.0,2.0,3.0,49.0,1.0,2.0,0.0,1.0
3,check_low,debt_onSchedule,pur_furniture,sav_small,emp_lessSeven,debtor_guarantor,prop_agreement,oi_none,hous_free,job_skilled,...,1,42.0,7882.0,2.0,4.0,45.0,NaN,2.0,0.0,1.0
4,check_low,debt_delay,pur_newCar,sav_small,emp_lessFour,debtor_none,prop_none,oi_none,hous_free,job_skilled,...,1,24.0,4870.0,3.0,NaN,53.0,2.0,2.0,0.0,1.0


In [7]:
import math

df['test'] = 2*df['gender'] + (df['duration'] / 1000)

In [10]:
np.ceil(df['test'])

0      3.0
1      1.0
2      3.0
3      3.0
4      3.0
      ... 
995    1.0
996    3.0
997    3.0
998    3.0
999    3.0
Name: test, Length: 1000, dtype: float64